In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from pandas import read_csv, DataFrame

# Paths to image folders and label files
data_dir = '/content/drive/MyDrive/Colab Notebooks/birds'

# Paths to image folders and label files
train_img_dir = data_dir + '/Train'
test_img_dir = data_dir + '/Test'
train_labels_file = data_dir + '/train.txt'
test_labels_file = data_dir + '/test.txt'

# Load the train and test label files
train_labels: DataFrame = read_csv(train_labels_file, sep=" ", header=None, names=['filename', 'label'])
test_labels: DataFrame = read_csv(test_labels_file, sep=" ", header=None, names=['filename', 'label'])

# Add paths to the images
train_labels['filepath'] = train_labels['filename'].apply(lambda x: os.path.join(train_img_dir, x))
test_labels['filepath'] = test_labels['filename'].apply(lambda x: os.path.join(test_img_dir, x))

# Display the first few rows
print(train_labels.head())

                                     filename  label  \
0  Black_footed_Albatross_0004_2731401028.jpg      0   
1  Black_footed_Albatross_0008_1384283201.jpg      0   
2   Black_footed_Albatross_0011_820118444.jpg      0   
3  Black_footed_Albatross_0003_2981373810.jpg      0   
4  Black_footed_Albatross_0021_2443213385.jpg      0   

                                            filepath  
0  /content/drive/MyDrive/Colab Notebooks/birds/T...  
1  /content/drive/MyDrive/Colab Notebooks/birds/T...  
2  /content/drive/MyDrive/Colab Notebooks/birds/T...  
3  /content/drive/MyDrive/Colab Notebooks/birds/T...  
4  /content/drive/MyDrive/Colab Notebooks/birds/T...  


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle

In [4]:
# Parameters
img_size = (224, 224)  # Image size for ResNet50 input
batch_size = 32
num_classes = 200

# Function to load images from filepaths and resize them
def load_and_preprocess_images(df, img_size):
    datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_dataframe(
        dataframe=df,
        x_col='filepath',
        y_col='label',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='raw',  # Raw mode for label mapping as integers
        shuffle=True
    )
    return generator

# Shuffle the data to ensure randomness
train_labels = shuffle(train_labels)
test_labels = shuffle(test_labels)

# Preprocess images for training and testing
train_generator = load_and_preprocess_images(train_labels, img_size)
test_generator = load_and_preprocess_images(test_labels, img_size)

Found 4829 validated image filenames.
Found 1156 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 48 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [5]:
# Load pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(200, activation='relu')(x)
preds = Dense(num_classes, activation='softmax')(x)

# Build the full model
model = Model(inputs=base_model.input, outputs=preds)

# Freeze the layers of the base model to train only the custom layers first
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss='sparse_categorical_crossentropy',  # Since we have integer labels
    metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=6,
    validation_data=test_generator)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 1685s 11s/step - accuracy: 0.0043 - loss: 1475.0941 - val_accuracy: 0.0061 - val_loss: 5.3282
Epoch 2/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 41s 260ms/step - accuracy: 0.0059 - loss: 5.3528 - val_accuracy: 0.0061 - val_loss: 5.3315
Epoch 3/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 44s 280ms/step - accuracy: 0.0045 - loss: 5.3592 - val_accuracy: 0.0069 - val_loss: 5.3248
Epoch 4/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 79s 261ms/step - accuracy: 0.0030 - loss: 5.3465 - val_accuracy: 0.0061 - val_loss: 5.3208
Epoch 5/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 41s 261ms/step - accuracy: 0.0045 - loss: 5.3551 - val_accuracy: 0.0061 - val_loss: 5.3206
Epoch 6/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 40s 257ms/step - accuracy: 0.0036 - loss: 5.3475 - val_accuracy: 0.0061 - val_loss: 5.3224


In [6]:
# Fine-tune the model by unfreezing some layers
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_generator,
    epochs=6,
    validation_data=test_generator)

# Save the model for future use
model.save('bird_species_classifier.h5')

# Evaluate the model on the test set
eval_result = model.evaluate(test_generator)
print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1] * 100:.2f}%")

# Predict on test set
y_pred = np.argmax(model.predict(test_generator), axis=1)

# Get ground truth labels
y_true = test_generator.labels

# Top-1 Accuracy Calculation
top1_accuracy = np.mean(y_true == y_pred)
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")

Epoch 1/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 74s 355ms/step - accuracy: 0.0052 - loss: 9.8392 - val_accuracy: 0.0061 - val_loss: 5.4555
Epoch 2/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 64s 292ms/step - accuracy: 0.0053 - loss: 5.3251 - val_accuracy: 0.0061 - val_loss: 5.3747
Epoch 3/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 42s 268ms/step - accuracy: 0.0077 - loss: 5.3248 - val_accuracy: 0.0061 - val_loss: 5.3768
Epoch 4/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 82s 272ms/step - accuracy: 0.0044 - loss: 5.3227 - val_accuracy: 0.0061 - val_loss: 5.3862
Epoch 5/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 44s 283ms/step - accuracy: 0.0051 - loss: 5.3274 - val_accuracy: 0.0061 - val_loss: 5.3883
Epoch 6/6
151/151 ━━━━━━━━━━━━━━━━━━━━ 42s 271ms/step - accuracy: 0.0061 - loss: 5.3268 - val_accuracy: 0.0061 - val_loss: 5.3890


37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 222ms/step - accuracy: 0.0038 - loss: 5.4120
Test Loss: 5.388988971710205, Test Accuracy: 0.61%
37/37 ━━━━━━━━━━━━━━━━━━━━ 14s 291ms/step
Top-1 Accuracy: 0.61%
Average Accuracy per Class: nan%


<ipython-input-6-33a5d563f4f5>:44: RuntimeWarning: invalid value encountered in divide
  avg_accuracy_per_class = np.mean(class_correct / class_total)


In [8]:
# Average Accuracy per Class Calculation
class_correct = np.zeros(num_classes)
class_total = np.zeros(num_classes)

for i in range(len(y_true)):
    class_correct[y_true[i]] += y_true[i] == y_pred[i]
    class_total[y_true[i]] += 1

class_accuracy = np.divide(class_correct, class_total, out=np.zeros_like(class_correct), where=class_total!=0)
avg_accuracy_per_class = np.mean(class_accuracy)
print(f"Average Accuracy per Class: {avg_accuracy_per_class * 100:.2f}%")

Average Accuracy per Class: 0.50%
